In [1]:
import warnings
warnings.filterwarnings('ignore')
import phpserialize as php # 处理PHP序列化的数据
from tqdm import tqdm
import pandas as pd
import os
import re
import json

In [2]:
# 获取刊登数据
publish_data_path = list(os.walk('./Data_input/item-刊登'))[0][0] + '/' + list(os.walk('./Data_input/item-刊登'))[0][2][0]
publish_data_path_suffix = publish_data_path.split('.')[-1]
if publish_data_path_suffix == 'csv':
    publish_sku = pd.read_csv(publish_data_path,dtype=str).fillna('')
else:
    publish_sku = pd.read_excel(publish_data_path,dtype=str).fillna('')

publish_sku= publish_sku[publish_sku['listingstatus']=='Active'].copy()
publish_sku.reset_index(drop=True, inplace=True)
publish_sku = publish_sku[publish_sku['sku'].str.find('DEB-')==-1].copy()
# publish_sku

In [3]:
def decode_bytes(bytes_str): # 解码
    if isinstance(bytes_str, dict):
        return {decode_bytes(k): decode_bytes(v) for k, v in bytes_str.items()}
    elif isinstance(bytes_str, list):
        return [decode_bytes(item) for item in bytes_str]
    elif isinstance(bytes_str, bytes):
        return bytes_str.decode('utf-8')
    else:
        return bytes_str

def filter_list(ls): # 过滤列表中不存在数字部分
    # pattern = re.compile(r'^[a-zA-Z\s]+$') # 纯英文
    pattern = re.compile(r'\d')
    return [item for item in ls if pattern.search(item)]

def process_row(row): # 对每行的php序列化数据进行处理
    php_serialized_str = row['itemspecifics']
    ItemSpecific_data = php.loads(php_serialized_str.encode('utf-8'))
    decoded_data = decode_bytes(ItemSpecific_data)
    
    df_ItemSpecific = pd.DataFrame(decoded_data['NameValueList'])
    df_ItemSpecific.columns = df_ItemSpecific.iloc[0]
    df_ItemSpecific = df_ItemSpecific[1:]
    
    # 分离出带有特定关键词的列生成新df，用于合并形成完整oe
    keyword_cols = df_ItemSpecific.filter(regex='(?i)number|oe|nummer|numéro|Produktgruppe').fillna('')
    
    # 处理含上述关键词的新df，将数据整合成字符串
    oe_publish = []
    for col in keyword_cols:
        try:
            oe_publish.append(';'.join(filter_list(keyword_cols[col].values.tolist())))
        except:
            oe_values = [str(val) for val in keyword_cols[col].values.tolist()]
            oe_publish.append(';'.join(filter_list(oe_values)))
    oe_publish = ';'.join([oe for oe in oe_publish if oe])
    return pd.Series({'OE_刊登': oe_publish})



def process_row(row): # 对每行的php序列化数据进行处理
    php_serialized_str = row['itemspecifics']
    try:
        ItemSpecific_data = php.loads(php_serialized_str.encode('utf-8'))
        decoded_data = decode_bytes(ItemSpecific_data)
        df_ItemSpecific = pd.DataFrame(decoded_data['NameValueList'])
        df_ItemSpecific.columns = df_ItemSpecific.iloc[0]
        df_ItemSpecific = df_ItemSpecific[1:2]
    except:
        decoded_data = decode_bytes(str(php_serialized_str))
        df_ItemSpecific = pd.DataFrame()
  
    # ItemSpecific_data = php.loads(php_serialized_str.encode('utf-8'))
    # decoded_data = decode_bytes(ItemSpecific_data)
    # df_ItemSpecific = pd.DataFrame(decoded_data['NameValueList'])
    # df_ItemSpecific.columns = df_ItemSpecific.iloc[0]
    # df_ItemSpecific = df_ItemSpecific[1:2]
    
    # 分离出带有特定关键词的列生成新df，用于合并形成完整oe
    keyword_cols = df_ItemSpecific.filter(regex='(?i)number|oe|nummer|numéro|Produktgruppe').fillna('')
    # 处理含上述关键词的新df，将数据整合成字符串
    oe_publish = []
    for col in keyword_cols:
        try:
            oe_publish.append(';'.join(filter_list(keyword_cols[col].values.tolist())))
        except:
            oe_values = [str(val[1]) for val in keyword_cols[col].values.tolist()[0].items()]
            oe_publish.append(';'.join(filter_list(oe_values)))
    oe_publish = ';'.join([oe for oe in oe_publish if oe])
    df_ItemSpecific['OE_刊登'] = oe_publish
    return df_ItemSpecific[['OE_刊登']] # 只返回新生成的列

def process_in_batches(df, batch_size):
    result_dfs = []
    for batch_start in tqdm(range(0, len(df), batch_size), desc='Processing Batches', ncols=100):
        batch_end = batch_start + batch_size
        df_batch = df.iloc[batch_start:batch_end]

        # 处理每个批次
        batch_results = df_batch.apply(process_row, axis=1)
        batch_results_df = pd.concat(batch_results.tolist(), ignore_index=True)
        result_dfs.append(batch_results_df)

    return pd.concat(result_dfs, ignore_index=True)

In [4]:
# 得到解码后合并的刊登oe
if __name__ == "__main__":
    batch_size = 50 # 50个一组（分组处理加快速度）
    publish_sku_processed = process_in_batches(publish_sku, batch_size)
    publish_sku_combined = pd.concat([publish_sku.reset_index(drop=True), publish_sku_processed.reset_index(drop=True)], axis=1)
# 删除所有括号及括号内所有内容
publish_sku_combined['OE_刊登'] = publish_sku_combined['OE_刊登'].apply(lambda x:re.sub(r'[(（][^)）]*[)）]', '', str(x)))
publish_sku_combined['OE_刊登'] = publish_sku_combined['OE_刊登'].apply(lambda x:str(x).replace(r'nan', ''))
publish_sku_combined

Processing Batches: 100%|█████████████████████████████████████████| 311/311 [00:33<00:00,  9.22it/s]


,country,site,selleruserid,itemid,sku,listingstatus,itemspecifics,OE_刊登
0,DE,Austria,premium-autoteile,355943105598,P-DEWP72350-FC,Active,"a:1:{s:13:""NameValueList"";a:13:{i:0;a:3:{s:4:""...",2712000401;41012;2712001001;147-2350;AW6030;W1...
1,US,eBayMotors,a-premium-us,126616531051,AP-USRAD02781+USCON34900,Active,"a:1:{s:13:""NameValueList"";a:26:{i:0;a:2:{s:4:""...",1640050340;8846030871;884600P020;2781;8012781;...
2,DE,Belgium_French,apm-autoteile,266936701968,DESA333311LR-C,Active,"a:1:{s:13:""NameValueList"";a:20:{i:0;a:3:{s:4:""...","312107,312108,333310,333311,543024Z001,543024Z..."
3,DE,Germany,apm-autoteile,266938913531,DEABS20241-2-FC,Active,"a:1:{s:13:""NameValueList"";a:20:{i:0;a:2:{s:4:""...","479507427R, 0265009929, 61273, 30224, 82901259..."
4,DE,France,premium-autoteile,256599024136,P-DETH01625,Active,"a:1:{s:13:""NameValueList"";a:15:{i:0;a:3:{s:4:""...","11617790524, 11617805437, V20-1625"
...,...,...,...,...,...,...,...,...
15507,DE,Ireland,carparts_premium,204932941838,Car-DECVA41180+DECVA42180,Active,"a:1:{s:13:""NameValueList"";a:19:{i:0;a:3:{s:4:""...",18-142180;4473305400;4473302000;4473305000;A44...
15508,DE,Austria,carparts_premium,204932941837,Car-DESA339081LR-C+DECSH2114,Active,"a:1:{s:13:""NameValueList"";a:15:{i:0;a:3:{s:4:""...","313881, 313882, 339080, 339081, 4060A173, 4060..."
15509,DE,Ireland,carparts_premium,196552735312,Car-DEEM10045-C,Active,"a:1:{s:13:""NameValueList"";a:16:{i:0;a:3:{s:4:""...","50830SDAA01, 50850SDBA00, 50870SDBA02, 50860SE..."
15510,DE,Italy,carparts_premium,196552735307,Car-DECVA41180+DECVA42180,Active,"a:1:{s:13:""NameValueList"";a:20:{i:0;a:3:{s:4:""...","18-142180, 4473305400, 4473302000, 4473305000,..."


In [5]:
publish_sku_combined['sku_匹配'] =  publish_sku_combined['sku'].copy()
# 删除所有括号及括号内所有内容
publish_sku_combined['sku_匹配'] = publish_sku_combined['sku_匹配'].apply(lambda x:re.sub(r'[(（][^)）]*[)）]', '', x))
for i in range(5): #以"%%%-"(%表单字符串)开头的内容删掉，循环五次，删除干净（例如P-DEB-sku）
    publish_sku_combined['sku_匹配'] = publish_sku_combined['sku_匹配'].apply(lambda x: re.sub(r'^[^-]*-', '', x) if ('-' in x and len(x.split('-')[0]) < 4) else x)
# 删除"DEB-"
publish_sku_combined['sku_匹配'] = publish_sku_combined['sku_匹配'].apply(lambda x:x.replace('DEB-', '').replace('-de', ''))

def process_sku1(row):
    if row['country'] != 'DE':
        return row['sku_匹配'][2:].replace('+US', '+').replace('+UK', '+').replace('+BM', '+')
    elif row['sku_匹配'][:2] == 'DE':
        return row['sku_匹配'][2:].replace('+DE', '+')
    elif row['site'] == 'France' and row['sku_匹配'][:2] == 'FR':
        return row['sku_匹配'][2:].replace('+FR', '+').replace('+DEB', '+')
    else:
        return row['sku_匹配'].replace('+DEB', '+')

def process_sku2(r):
    if r[-2:] == '-C':
        return re.sub(r'[^a-zA-Z0-9]+[a-zA-Z0-9]*$', '', r[:-2]) + '-C'
    elif r[-3:] == '-FC':
        return re.sub(r'[^a-zA-Z0-9]+[a-zA-Z0-9]*$', '', r[:-3]) + '-FC'
    else:
        return re.sub(r'[^a-zA-Z0-9]+[a-zA-Z0-9]*$', '', r)

def process_sku3(row): #先分割加号再对每个sku进行后缀处理，嵌套两次处理已经足够（可以改成while加正则判断是否存在）
    if '+' in row['sku_匹配']:
        list_split_sku = [process_sku2(process_sku2(r)) for r in row['sku_匹配'].split('+')]
        return '+'.join(list_split_sku)
    else:
        return process_sku2(process_sku2(row['sku_匹配']))

publish_sku_combined['sku_匹配'] = publish_sku_combined.apply(process_sku1, axis=1)
publish_sku_combined['sku_匹配'] = publish_sku_combined.apply(process_sku3, axis=1)
publish_sku_combined['清理后刊登oe'] =  publish_sku_combined['OE_刊登'].copy().fillna('')

In [6]:
# oe号进一步清洗
def oe_clean(row):
    oe_publish_clean = re.sub('\（','(',row['清理后刊登oe']) #将所有中文"（"符号替换为英文"("
    oe_publish_clean = re.sub('\）',')',oe_publish_clean) #将所有中文"）"符号替换为英文")"
    oe_publish_clean = re.sub('\\([^)]*\\)','',oe_publish_clean)  #删除所有"()"及其内部的字符（因为通常不是OE号）
    oe_publish_clean = re.sub('[/ .\-!*+]', '', oe_publish_clean)  #删掉'/'、'-'、'.'、'!'、'*'和空格
    oe_publish_clean = re.sub('[,，]', ';', oe_publish_clean) #中英文逗号替换为英文分号
    oe_publish_clean = re.sub('\u200e', '', oe_publish_clean) #清除不可见字符
    list1 = [i for i in str(oe_publish_clean).split(';') if i != '' and i.isalpha() == False] #去除空值
    list1 = filter_list(list1) # 去除完全由非数字组成部分
    list1 = [re.sub(r'\r','',i) for i in list1]
    list2 = list(set(list1)) #去重
    list2.sort(key = list1.index)# 去重后保持OE号原来的顺序
    oe_publish_clean = ';'.join(list2)
    try:
        if oe_publish_clean[0] == ';':
            oe_publish_clean = oe_publish_clean[1:]
        elif oe_publish_clean[-1] == ';':
            oe_publish_clean = oe_publish_clean[:-1]
    except:
        pass
    return str(oe_publish_clean.upper())

publish_sku_combined['清理后刊登oe'] = publish_sku_combined.apply(oe_clean, axis=1)
publish_sku_combined

,country,site,selleruserid,itemid,sku,listingstatus,itemspecifics,OE_刊登,sku_匹配,清理后刊登oe
0,DE,Spain,apm-autoteile,266882282528,DEB-CA48922-FE3,Active,"a:1:{s:13:""NameValueList"";a:10:{i:0;a:3:{s:4:""...","415 266 00 00, 11220-5217R, A415 266 00 00, 11...",CA48922,4152660000;112205217R;A4152660000;1132000Q0D;1...
1,DE,Spain,apm-autoteile,256599048626,DEB-IMG54304-VIC3,Active,"a:1:{s:13:""NameValueList"";a:15:{i:0;a:3:{s:4:""...","1 146 053, SU001-00497, 0209.AH, Y401-10-2H1",IMG54304,1146053;SU00100497;0209AH;Y401102H1
2,DE,France,premium-autoteile,256581208645,P-DEB-CA03733-FE3,Active,"a:1:{s:13:""NameValueList"";a:15:{i:0;a:3:{s:4:""...","103733;33326792525, 33326792525S1",CA03733,103733;33326792525;33326792525S1
3,DE,Austria,apm-autoteile,256599021508,DEB-IMG72000-AJU3,Active,"a:1:{s:13:""NameValueList"";a:13:{i:0;a:3:{s:4:""...",10072000;AJU10072000;1111554030;1111554020;111...,IMG72000,10072000;AJU10072000;1111554030;1111554020;111...
4,DE,Austria,premium-autoteile,355943105598,P-DEWP72350-FC,Active,"a:1:{s:13:""NameValueList"";a:13:{i:0;a:3:{s:4:""...",2712000401;41012;2712001001;147-2350;AW6030;W1...,WP72350-FC,2712000401;41012;2712001001;1472350;AW6030;W1974M
...,...,...,...,...,...,...,...,...,...,...
56754,DE,Ireland,carparts_premium,196552735312,Car-DEEM10045-C,Active,"a:1:{s:13:""NameValueList"";a:16:{i:0;a:3:{s:4:""...","50830SDAA01, 50850SDBA00, 50870SDBA02, 50860SE...",EM10045-C,50830SDAA01;50850SDBA00;50870SDBA02;50860SEPA0...
56755,DE,Italy,carparts_premium,196552735307,Car-DECVA41180+DECVA42180,Active,"a:1:{s:13:""NameValueList"";a:20:{i:0;a:3:{s:4:""...","18-142180, 4473305400, 4473302000, 4473305000,...",CVA41180+CVA42180,18142180;4473305400;4473302000;4473305000;A447...
56756,DE,Ireland,carparts_premium,196552735305,Car-DEBCN41984C-FC+DEBCN41985C-FC,Active,"a:1:{s:13:""NameValueList"";a:18:{i:0;a:3:{s:4:""...","1;341984, 542245, 90421744, 429831;341985, 429...",BCN41984C-FC+BCN41985C-FC,1;341984;542245;90421744;429831;341985;429832;...
56757,DE,Austria,carparts_premium,196552735302,Car-DEHB13323-C,Active,"a:1:{s:13:""NameValueList"";a:14:{i:0;a:3:{s:4:""...","513323;31206850158;VKBA6669;513323, 3120685015...",HB13323-C,513323;31206850158;VKBA6669


In [7]:
# 与之前全量数据合并
before_all_data_path = list(os.walk('./Data_input/全上架ID'))[0][0] + '/' + list(os.walk('./Data_input/全上架ID'))[0][2][0]
before_all_data = pd.read_csv(before_all_data_path, dtype=str).fillna('')
used_all_data = pd.concat([before_all_data, publish_sku_combined], axis=0)
# 更新数据
used_all_data.drop_duplicates(subset=['itemid'],keep='last',inplace=True)

In [8]:
# 获取下架数据
remove_data_path = list(os.walk('./Data_input/item-下架'))[0][0] + '/' + list(os.walk('./Data_input/item-下架'))[0][2][0]
remove_data_path_suffix = publish_data_path.split('.')[-1]
if remove_data_path_suffix == 'csv':
    remove_sku = pd.read_csv(remove_data_path,dtype=str).fillna('')
else:
    remove_sku = pd.read_excel(remove_data_path,dtype=str).fillna('')
remove_listingstatus = remove_sku[['itemid','listingstatus']]
remove_listingstatus.rename(columns={'listingstatus':'状态更新'},inplace=True)
# remove_listingstatus

In [9]:
# 匹配进全量数据
used_all_data = pd.merge(used_all_data, remove_listingstatus, on='itemid', how='left').fillna('')
# 更新状态
mask = (used_all_data['状态更新'] != '') & (used_all_data['状态更新'] != 'Active')  
used_all_data.loc[mask, 'listingstatus'] = used_all_data.loc[mask, '状态更新']
# 得到最终全量数据
all_data = used_all_data[used_all_data['listingstatus']=='Active'].copy()
all_data.to_csv('./Data_output/【已更新】SKU_publish.csv', index=False)
all_data = all_data[all_data['sku'].str.find('DEB-')==-1].copy()
all_data

,Unnamed: 0,country,site,selleruserid,itemid,sku,listingstatus,itemspecifics,OE_刊登,sku_匹配,清理后刊登oe,状态更新
0,0,DE,Italy,carparts_premium,203235952319.0,Car-DECAKBM018,Active,"a:1:{s:13:""NameValueList"";a:13:{i:0;a:3:{s:4:""...","32111139316, 32111139315, 31351091764",CAKBM018,32111139316;32111139315;31351091764,
1,1,DE,France,carparts_premium,203235952705.0,Car-DEDLA01935,Active,"a:1:{s:13:""NameValueList"";a:17:{i:0;a:3:{s:4:""...","2047201935,2047200735;2047201935,2047200735",DLA01935,2047201935;2047200735,
2,2,DE,Italy,carparts_premium,203235952614.0,Car-DEDLA37849,Active,"a:1:{s:13:""NameValueList"";a:18:{i:0;a:3:{s:4:""...","51217185692, 51217155154",DLA37849,51217185692;51217155154,
3,3,DE,Italy,carparts_premium,203235952388.0,Car-DEDLA85687,Active,"a:1:{s:13:""NameValueList"";a:16:{i:0;a:3:{s:4:""...",51227185687,DLA85687,51227185687,
4,4,DE,Spain,carparts_premium,203235952511.0,Car-DEDLA01935,Active,"a:1:{s:13:""NameValueList"";a:16:{i:0;a:3:{s:4:""...","2047201935,2047200735;2047201935,2047200735",DLA01935,2047201935;2047200735,
...,...,...,...,...,...,...,...,...,...,...,...,...
2057116,,DE,Ireland,carparts_premium,196552735312,Car-DEEM10045-C,Active,"a:1:{s:13:""NameValueList"";a:16:{i:0;a:3:{s:4:""...","50830SDAA01, 50850SDBA00, 50870SDBA02, 50860SE...",EM10045-C,50830SDAA01;50850SDBA00;50870SDBA02;50860SEPA0...,
2057117,,DE,Italy,carparts_premium,196552735307,Car-DECVA41180+DECVA42180,Active,"a:1:{s:13:""NameValueList"";a:20:{i:0;a:3:{s:4:""...","18-142180, 4473305400, 4473302000, 4473305000,...",CVA41180+CVA42180,18142180;4473305400;4473302000;4473305000;A447...,
2057118,,DE,Ireland,carparts_premium,196552735305,Car-DEBCN41984C-FC+DEBCN41985C-FC,Active,"a:1:{s:13:""NameValueList"";a:18:{i:0;a:3:{s:4:""...","1;341984, 542245, 90421744, 429831;341985, 429...",BCN41984C-FC+BCN41985C-FC,1;341984;542245;90421744;429831;341985;429832;...,
2057119,,DE,Austria,carparts_premium,196552735302,Car-DEHB13323-C,Active,"a:1:{s:13:""NameValueList"";a:14:{i:0;a:3:{s:4:""...","513323;31206850158;VKBA6669;513323, 3120685015...",HB13323-C,513323;31206850158;VKBA6669,
